# Importing libraries

In [7]:
from ast import literal_eval
import pandas as pd
import numpy as np

make functions

In [56]:
# Calculate Picker Route Distance between two locations
def distance_picking(Loc1, Loc2, y_low=1, y_high=20):

    # Start Point
    x1, y1 = Loc1[0], Loc1[1]
    # End Point
    x2, y2 = Loc2[0], Loc2[1]
    # Distance x-axis
    distance_x = abs(x2 - x1)
    # Distance y-axis
    if x1 == x2:
        distance_y1 = abs(y2 - y1)
        distance_y2 = distance_y1
    else:
        distance_y1 = (y_high - y1) + (y_high - y2)
        distance_y2 = (y1 - y_low) + (y2 - y_low)
    # Minimum distance on y-axis 
    distance_y = min(distance_y1, distance_y2)    
    # Total distance
    distance = distance_x + distance_y

    return distance

# Find closest next location 
def next_location(start_loc, list_locs, list_vol, list_dest, y_low, y_high):

    # Distance to every next points candidate
    list_dist = [distance_picking(start_loc, i, y_low, y_high) for i in list_locs]
    # Minimum Distance 
    distance_next = min(list_dist)
    # Location of minimum distance
    index_min = list_dist.index(min(list_dist))
    next_vol = list_vol[index_min]   # Follow the index of the location into the volume index
    list_vol.pop(index_min)          # Next volume is removed from the list of candidates
    next_loc = list_locs[index_min]  # Next location is the first location with distance = min (**)
    list_locs.pop(index_min)         # Next location is removed from the list of candidates
    next_dest = list_dest[index_min] # Next destination is the first location with distance = min (**)
    list_dest.pop(index_min)         # Next destination is removed from the list of candidates

    
    return list_locs, start_loc, next_loc, next_vol, next_dest, distance_next

# Calculate total distance to cover for a list of locations
def create_picking_route(origin_loc, list_locs, list_vol, list_dest, y_low, y_high):
    
    # Start volume
    volume_start = 0
    # Total distance variable
    wave_distance = 0
    # Current location variable 
    start_loc = origin_loc
    # Start destination
    dest_start = 0
    
    # Store routes & volume
    list_volume = []
    list_volume.append(volume_start)
    list_chemin = []
    list_chemin.append(start_loc)
    list_destination = []
    list_destination.append(dest_start)
    
    while len(list_locs) > 0 & len(list_vol): # Looping until all locations are picked
        # Going to next location
        list_locs, start_loc, next_loc, next_vol, next_dest, distance_next = next_location(start_loc, list_locs, list_vol, list_dest, y_low, y_high)
        # Update start_loc & volume position
        start_loc = next_loc
        list_chemin.append(start_loc)
        list_volume.append(next_vol)
        list_destination.append(next_dest)
        # Update distance
        wave_distance = wave_distance + distance_next 

    # Final distance from last storage location to origin
    wave_distance = wave_distance + distance_picking(start_loc, origin_loc, y_low, y_high)
    list_chemin.append(origin_loc)
    list_volume.append(volume_start)
    
    return wave_distance, list_chemin, list_volume, list_destination

import the file

In [57]:
# Import dataframe with orderlines
df_orderlines = pd.read_excel('pick.xlsx')
df_orderlines

,No.,Coord,Volume,Location,x,y
0,1,"[1,5]",44,A5,1,5
1,2,"[1,10]",49,A10,1,10
2,3,"[2,11]",61,B11,2,11
3,4,"[2,5]",55,B5,2,5
4,5,"[1,11]",50,A11,1,11
5,6,"[2,9]",59,B9,2,9
6,7,"[1,6]",45,A6,1,6
7,8,"[2,8]",58,B8,2,8
8,9,"[2,3]",53,B3,2,3
9,10,"[2,6]",56,B6,2,6


make the columns to list

In [58]:
list_locs = list(df_orderlines['Coord'].apply(lambda t: literal_eval(t)).values)
list_vol = list(df_orderlines["Volume"])
list_dest = list(df_orderlines["Location"])

call the function

In [60]:
distance, org_loc, org_vol, org_dest = create_picking_route([0,0], list_locs, list_vol, list_dest, 1, 20)

In [61]:
df_org = pd.DataFrame({'location':org_loc, 'volume':org_vol})

#remove the first and last(loc[0,0])
df_org_loc = df_org[1:-1]
df_org_loc.reset_index(inplace=True, drop=True)
org_dest = org_dest[1:]
df_org_loc['destination'] = org_dest
df_org_loc

c:\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,location,volume,destination
0,"[1, 1]",40,A1
1,"[1, 2]",41,A2
2,"[1, 3]",42,A3
3,"[1, 4]",43,A4
4,"[1, 5]",44,A5
5,"[1, 6]",45,A6
6,"[1, 7]",46,A7
7,"[1, 8]",47,A8
8,"[1, 9]",48,A9
9,"[1, 10]",49,A10
